<a href="https://colab.research.google.com/github/AluminumShark/2024_Deep_Learning_with_Pytorch/blob/main/02_Convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'DEVICE: {device}')

EPOCHS = 128
BATCH_SIZE = 32
LEARNING_RATE = 0.01

print(f'EPOCHS: {EPOCHS}')
print(f'BATCH_SIZE: {BATCH_SIZE}')
print(f'LEARNING_RATE: {LEARNING_RATE}')

USING: cpu
EPOCHS: 128
BATCH_SIZE: 32
LEARNING_RATE: 0.01


In [11]:
# Load training into a single batch to compute mean and std

trainset = CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

data = next(iter(trainloader))

mean = data[0].mean()
std = data[0].std()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Create a Sequential Model

model = nn.Sequential(
    nn.Conv2d(3, 64, 5, stride=2, padding='same'),
    nn.BatchNorm2d(64),
    nn.ReLU(),

    nn.Conv2d(64, 64, 3, stride=2, padding='same'),
    nn.BatchNorm2d(64),
    nn.ReLU()

)